In [1]:
# Real estate price prediction project

In [2]:
!pip install -U autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter co

In [3]:
import pandas as pd
from autogluon.tabular import TabularPredictor
import joblib

In [4]:
#read data
df=pd.read_csv("kaggle source.csv")

In [5]:
#EDA
df.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [6]:
df.shape

(414, 8)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   No                                      414 non-null    int64  
 1   X1 transaction date                     414 non-null    float64
 2   X2 house age                            414 non-null    float64
 3   X3 distance to the nearest MRT station  414 non-null    float64
 4   X4 number of convenience stores         414 non-null    int64  
 5   X5 latitude                             414 non-null    float64
 6   X6 longitude                            414 non-null    float64
 7   Y house price of unit area              414 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB


In [8]:
df.corr()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
No,1.000000,-0.048658,-0.032808,-0.013573,-0.012699,-0.010110,-0.011059,-0.028587
X1 transaction date,-0.048658,1.000000,0.017549,0.060880,0.009635,0.035058,-0.041082,0.087491
X2 house age,-0.032808,0.017549,1.000000,0.025622,0.049593,0.054420,-0.048520,-0.210567
X3 distance to the nearest MRT station,-0.013573,0.060880,0.025622,1.000000,-0.602519,-0.591067,-0.806317,-0.673613
X4 number of convenience stores,-0.012699,0.009635,0.049593,-0.602519,1.000000,0.444143,0.449099,0.571005
X5 latitude,-0.010110,0.035058,0.054420,-0.591067,0.444143,1.000000,0.412924,0.546307
X6 longitude,-0.011059,-0.041082,-0.048520,-0.806317,0.449099,0.412924,1.000000,0.523287
Y house price of unit area,-0.028587,0.087491,-0.210567,-0.673613,0.571005,0.546307,0.523287,1.000000


In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
No,414.0,207.500000,119.655756,1.00000,104.250000,207.50000,310.750000,414.00000
X1 transaction date,414.0,2013.148971,0.281967,2012.66700,2012.917000,2013.16700,2013.417000,2013.58300
X2 house age,414.0,17.712560,11.392485,0.00000,9.025000,16.10000,28.150000,43.80000
X3 distance to the nearest MRT station,414.0,1083.885689,1262.109595,23.38284,289.324800,492.23130,1454.279000,6488.02100
X4 number of convenience stores,414.0,4.094203,2.945562,0.00000,1.000000,4.00000,6.000000,10.00000
X5 latitude,414.0,24.969030,0.012410,24.93207,24.963000,24.97110,24.977455,25.01459
X6 longitude,414.0,121.533361,0.015347,121.47353,121.528085,121.53863,121.543305,121.56627
Y house price of unit area,414.0,37.980193,13.606488,7.60000,27.700000,38.45000,46.600000,117.50000


In [10]:
df.isnull().sum()

,0
No,0
X1 transaction date,0
X2 house age,0
X3 distance to the nearest MRT station,0
X4 number of convenience stores,0
X5 latitude,0
X6 longitude,0
Y house price of unit area,0


In [12]:
#modeling

In [13]:
target = 'Y house price of unit area'

In [17]:
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

In [20]:
predictor = TabularPredictor(
    label=target,
    problem_type="regression",
    eval_metric="rmse"
).fit(
    df,
    presets="medium_quality_faster_train",
    time_limit=120   # saniye
)

No path specified. Models will be saved in: "AutogluonModels/ag-20260120_185019"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cpu
CUDA Version:       CUDA is not available
Memory Avail:       8.86 GB / 12.67 GB (69.9%)
Disk Space Avail:   85.56 GB / 107.72 GB (79.4%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save models to "/content/AutogluonModels/ag-20260120_185019"
Train Data Rows:    414
Train Data Columns: 7
Label Column:       Y house price of unit area
Problem Type:       regression
Preprocessing data ...
Using Feature Generat

In [21]:
predictor.evaluate(test_data)

{'root_mean_squared_error': np.float64(-7.760135608146655),
 'mean_squared_error': -60.21970465682565,
 'mean_absolute_error': -4.372585110492017,
 'r2': 0.7549351854851747,
 'pearsonr': 0.8819933696724063,
 'median_absolute_error': np.float64(-3.0251724243164055)}

In [22]:
predictor.leaderboard(test_data, silent=True) #compare models

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesMSE,-4.172885,-6.598157,root_mean_squared_error,0.144408,0.148397,1.129063,0.144408,0.148397,1.129063,1,True,5
1,RandomForestMSE,-4.261246,-6.898959,root_mean_squared_error,0.161058,0.166849,1.322951,0.161058,0.166849,1.322951,1,True,3
2,XGBoost,-6.016260,-7.027314,root_mean_squared_error,0.021673,0.004474,0.883061,0.021673,0.004474,0.883061,1,True,7
3,LightGBMLarge,-6.019558,-6.846405,root_mean_squared_error,0.015331,0.003311,1.138234,0.015331,0.003311,1.138234,1,True,9
4,CatBoost,-6.249460,-6.560746,root_mean_squared_error,0.018548,0.001661,1.364108,0.018548,0.001661,1.364108,1,True,4
5,WeightedEnsemble_L2,-7.760136,-6.214993,root_mean_squared_error,0.173030,0.179441,3.402267,0.005159,0.000653,0.030343,2,True,10
6,LightGBM,-8.026040,-6.362969,root_mean_squared_error,0.003710,0.001496,0.758266,0.003710,0.001496,0.758266,1,True,2
7,LightGBMXT,-8.704486,-6.255465,root_mean_squared_error,0.006812,0.011938,2.048973,0.006812,0.011938,2.048973,1,True,1
8,NeuralNetTorch,-9.392213,-6.981065,root_mean_squared_error,0.026197,0.027829,8.484247,0.026197,0.027829,8.484247,1,True,8
9,NeuralNetFastAI,-9.584828,-7.486716,root_mean_squared_error,0.022636,0.015837,1.048084,0.022636,0.015837,1.048084,1,True,6


In [24]:
#model save
joblib.dump(predictor,"model.joblib")

['model.joblib']